In [ ]:
# pdfs_module_integrated.py
# ────────────────────
# 1) DEPENDENCIAS (ejecuta esta celda en Colab)
# ────────────────────────────────────────────────────────────────
!apt-get update -q && apt-get install -y -q chromium-chromedriver
!pip install -q selenium pandas xlrd==1.2.0 requests PyPDF2

# ────────────────────────────────────────────────────────────────
# 2) IMPORTS
# ────────────────────────────────────────────────────────────────
import os, sys, glob, time, base64, json, re
import pathlib, json
from datetime import datetime, timedelta
from typing import List, Tuple

import requests
from PyPDF2 import PdfReader, PdfWriter        # (PdfReader solo por compat.)
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from IPython.display import Image, display

# ────────────────────────────────────────────────────────────────
# 3) CREDENCIALES / MAPEOS
# ────────────────────────────────────────────────────────────────
CFG_PATH = pathlib.Path("eforce_config_cron.json")

if not CFG_PATH.exists():
    raise FileNotFoundError(f"⚠️ Archivo no encontrado: {CFG_PATH}")

with open(CFG_PATH, "r", encoding="utf-8") as f:
    config = json.load(f)

usuario         = config.get("usuario", "")
contraseña      = config.get("contraseña", "")
driver_mapping  = config.get("driver_mapping", {})
convertapi_secret = config.get("convertapi_secret", "")

print("✅ Configuración cargada correctamente.")



# ────────────────────────────────────────────────────────────────
# 4) RUTAS LOCALES PARA DESCARGAS (en Colab son carpetas temporales)
# ────────────────────────────────────────────────────────────────
download_path   = "Compilados"
conversion_path = os.path.join(download_path, "conversion")
pdfs_path       = os.path.join(download_path, "pdfs")
for p in (download_path, conversion_path, pdfs_path):
    os.makedirs(p, exist_ok=True)

# ────────────────────────────────────────────────────────────────
# 5) SELENIUM (Chrome headless)
# ────────────────────────────────────────────────────────────────
chrome_options = webdriver.ChromeOptions()
prefs = {"download.default_directory": download_path}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
sys.path.insert(0, '/usr/lib/chromium-browser/chromedriver')

driver = webdriver.Chrome(options=chrome_options)

# ────────────────────────────────────────────────────────────────
# 6) UTILIDADES BÁSICAS
# ────────────────────────────────────────────────────────────────
def ver_pantalla(nombre="pantalla.png"):
    driver.save_screenshot(nombre)
    display(Image(nombre))

def esperar_archivo_descargado(nombre, timeout=30):
    inicio = time.time()
    while time.time() - inicio < timeout:
        ruta = os.path.join(download_path, nombre)
        if os.path.exists(ruta):
            return ruta
        time.sleep(0.5)
    raise TimeoutException(f"⏳ Tiempo agotado esperando {nombre}")

def get_fecha_pedidos():
    hoy = datetime.today()
    dias = 2 if hoy.weekday() == 0 else 1      # lunes → retrocede 2 días
    return (hoy - timedelta(days=dias)).strftime("%Y-%m-%d")

# ────────────────────────────────────────────────────────────────
# 7) LOGIN
# ────────────────────────────────────────────────────────────────
def iniciar_sesion(driver):
    driver.get("https://myeforce.ecom.com.co/ecomltda/ingresar.php")
    time.sleep(2)
    driver.find_element(By.ID,"cta").send_keys(usuario)
    driver.find_element(By.ID,"ingr").send_keys(contraseña + Keys.RETURN)
    time.sleep(5)
    print("✅ Sesión iniciada correctamente.")

# ────────────────────────────────────────────────────────────────
# 8) LEER TABLA DE “PLANILLAS”  (limite = 20)
# ────────────────────────────────────────────────────────────────
def obtener_planillas(drv, limite: int = 20) -> List[Tuple[str, int]]:
    # 1) Abrir módulo con el límite deseado
    drv.get(f"https://myeforce.ecom.com.co/eforce/modulos/planillasabono2/index.php?limit={limite}")
    # 2) Esperar que aparezca el botón «Buscar» y hacer clic
    WebDriverWait(drv, 15).until(EC.element_to_be_clickable((By.ID, "read"))).click()

    # 3) Esperar a que la tabla de resultados se renderice
    WebDriverWait(drv, 15).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "table.table tbody tr"))
    )

    # 4) Extraer pares (planilla, idDriver)
    pares = []
    for fila in drv.find_elements(By.CSS_SELECTOR, "table.table tbody tr"):
        celdas = fila.find_elements(By.TAG_NAME, "td")
        if len(celdas) < 9:                   # descarta totales o filas vacías
            continue
        planilla  = celdas[7].text.strip()
        conductor = celdas[8].text.strip()    # ej: "108 - NHQ677-…"
        m = re.match(r"(\d{1,3})\s*-", conductor)
        if planilla.isdigit() and m:
            pares.append((planilla, int(m.group(1))))

    print(f"🔎 Planillas encontradas: {len(pares)}")
    return pares


# ────────────────────────────────────────────────────────────────
# 8) ========  FUNCIONES ORIGINALES (SIN CAMBIOS)  ===============
# ────────────────────────────────────────────────────────────────
def descargar_pedidos(driver, asesor_id, prefijo):
    try:
        driver.get("https://myeforce.ecom.com.co//eforce/modulos/pedidosNewpru2/index.php")
        WebDriverWait(driver,10).until(EC.presence_of_element_located((By.NAME,"ped_usuari")))
        Select(driver.find_element(By.NAME,"ped_usuari")).select_by_value(asesor_id)
        driver.execute_script("document.getElementById('b_fecha1').value=arguments[0];", get_fecha_pedidos())
        driver.execute_script("document.getElementsByName('ped_estado')[0].value='1';")
        driver.execute_script("document.getElementsByName('limit')[0].value='1000';")
        driver.find_element(By.ID,"read").click()
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"{prefijo}_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ pedidos {prefijo}: {e}")

def descargar_facturas(driver, asesor_id, prefijo):
    try:
        driver.get("https://myeforce.ecom.com.co//eforce/modulos/informes/vtafacturas/index.php")
        Select(driver.find_element(By.ID,"select_0")).select_by_value("10059830")
        time.sleep(1)
        driver.execute_script(
            "document.getElementsByName('fecha1')[0].value=arguments[0];",
            datetime.today().strftime("%Y-%m-%d")
        )
        Select(driver.find_element(By.ID,"select_1")).select_by_value(asesor_id)
        driver.find_element(By.XPATH,"//input[@value='Generar Informe']").click()
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"{prefijo}_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ facturas {prefijo}: {e}")

def descargar_entregas(driver):
    try:
        url = f"https://myeforce.ecom.com.co//eforce/modulos/entrega/index.php?modo=detalles&pla_numero={planilla_id}"
        driver.get(url); time.sleep(2)
        link = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,"//a[contains(@href,'.xls')]")))
        servidor = os.path.basename(link.get_attribute("href"))
        link.click()
        src = esperar_archivo_descargado(servidor)
        dst = f"Entregas_{planilla_id}.xls"
        os.rename(src, os.path.join(download_path, dst))
        print(f"📥 {dst}")
    except Exception as e:
        print(f"❌ entregas: {e}")

def convertir_a_csv_convertapi():
    url = "https://v2.convertapi.com/convert/xls/to/csv"
    headers = {"Authorization": f"Bearer {convertapi_secret}"}
    archivos = glob.glob(os.path.join(download_path,"*.xls"))
    if not archivos:
        print("⚠️ No hay .xls para convertir")
        return
    print("\n🔄 ConvertAPI XLS -> CSV")
    for xls in archivos:
        name = os.path.basename(xls)
        csv_name = name.replace(".xls",".csv")
        csv_path = os.path.join(conversion_path, csv_name)
        raw = open(xls,"rb").read()
        b64 = base64.b64encode(raw).decode()
        payload = {
            "Parameters":[
                {"Name":"File","FileValue":{"Name":name,"Data":b64}},
                {"Name":"StoreFile","Value":True}
            ]
        }
        r = requests.post(url, headers=headers, json=payload)
        if r.ok:
            csv_url = r.json()["Files"][0]["Url"]
            r2 = requests.get(csv_url)
            if r2.ok:
                open(csv_path,"wb").write(r2.content)
                print(f"✅ {csv_name}")
            else:
                print(f"❌ error descargar CSV {csv_name}")
        else:
            print(f"❌ ConvertAPI {name}: {r.text}")

# Helpers para PDFs
def obtener_sesion_requests(driver):
    s = requests.Session()
    for c in driver.get_cookies():
        s.cookies.set(c['name'], c['value'], domain=c.get('domain'))
    return s

def descargar_pdf_por_request(driver,url,destino):
    sess = obtener_sesion_requests(driver)
    resp = sess.get(url,stream=True)
    if resp.status_code==200:
        with open(destino,"wb") as f:
            for chunk in resp.iter_content(1024):
                f.write(chunk)
        print(f"✅ PDF guardado por request: {os.path.basename(destino)}")
    else:
        print(f"❌ error {resp.status_code} en request: {url}")

def esperar_por_patron(directorio, patrones, timeout=30):
    inicio=time.time()
    while time.time()-inicio<timeout:
        for f in glob.glob(os.path.join(directorio,"*")):
            if any(p in os.path.basename(f) for p in patrones):
                return f
        time.sleep(0.5)
    return None

def descargar_pdf_selenium(driver,url,patrones):
    driver.get(url); time.sleep(3)
    try:
        btn = WebDriverWait(driver,15).until(
            EC.element_to_be_clickable((By.XPATH,"//button[@id='download']"))
        )
        btn.click()
        found = esperar_por_patron(pdfs_path,patrones,timeout=30)
        if found:
            print(f"✅ PDF Selenium: {os.path.basename(found)}")
            return found
    except Exception as e:
        print(f"⚠️ Selenium PDF falló: {e}")
    return None

def descargar_pdfs(driver):
    urls = {
        "doc":f"https://myeforce.ecom.com.co//eforce/modulos/separacion/index.php?modo=imprimir&pla_numero={planilla_id}",
        "doc (1)":f"https://myeforce.ecom.com.co//eforce/modulos/planillasabono2/index.php?modo=imprimir&pla_numero={planilla_id}"
    }
    res={}
    for key,out in [("doc","file1.pdf"),("doc (1)","file2.pdf")]:
        url = urls[key]
        path = descargar_pdf_selenium(driver,url,[key])
        if not path:
            fallback = os.path.join(pdfs_path,f"{key}.pdf")
            descargar_pdf_por_request(driver,url,fallback)
            path = fallback if os.path.exists(fallback) else None
        res[key]=path
    return res

def generar_payload(pdf_paths):
    # PDFs
    pdf_entries=[]
    for key,outname in [("doc","file1.pdf"),("doc (1)","file2.pdf")]:
        path = pdf_paths.get(key)
        if path and os.path.exists(path):
            raw = base64.b64encode(open(path,"rb").read()).decode()
            m=re.search(r"(JVBERi0x[\s\S]*)",raw)
            clean=m.group(1) if m else raw
            bstr=f"data:@file/pdf;base64,{clean}"
        else:
            bstr=""
        pdf_entries.append({"base64":bstr,"fileType":"application/pdf","fileName":outname})

    # CSV (eliminar BOM)
    def encode_csv(path):
        data=open(path,"rb").read()
        if data.startswith(b'\xef\xbb\xbf'):
            data=data[3:]
        return base64.b64encode(data).decode()

    sales=sorted(f for f in glob.glob(conversion_path+"/*.csv") if "SAsesor" in f)
    data =sorted(f for f in glob.glob(conversion_path+"/*.csv") if "PAsesor" in f)
    csvSales=[{"base64":f"data:@file/csv;base64,{encode_csv(f)}","fileType":"text/csv","fileName":f"sales{i}.csv"} for i,f in enumerate(sales,1)]
    csvData =[{"base64":f"data:@file/csv;base64,{encode_csv(f)}","fileType":"text/csv","fileName":f"data{i}.csv"}  for i,f in enumerate(data ,1)]

    addf=os.path.join(conversion_path,f"Entregas_{planilla_id}.csv")
    if os.path.exists(addf):
        add={"fileName":"additional.csv","base64":f"data:@file/csv;base64,{encode_csv(addf)}","fileType":"text/csv"}
    else:
        add={"fileName":"additional.csv","base64":"","fileType":"text/csv"}

    return {
        "pdfFiles":pdf_entries,
        "csvFilesSales":csvSales,
        "csvFiles":csvData,
        "additionalCsvFile":add,
        "idDriver":idDriver
    }

def imprimir_payload_truncado(payload,lim=30):
    p=json.loads(json.dumps(payload))
    for k in ["pdfFiles","csvFilesSales","csvFiles"]:
        for it in p[k]:
            s=it["base64"]; it["base64"]=s[:lim]+"..." if len(s)>lim else s
    s=p["additionalCsvFile"]["base64"]
    p["additionalCsvFile"]["base64"]=s[:lim]+"..." if len(s)>lim else s
    print(json.dumps(p,indent=4))

def limpiar_archivos():
    for pattern in ("*.xls","*.csv","*.pdf"):
        for f in glob.glob(os.path.join(download_path,"*"+pattern.split('.')[-1])):
            os.remove(f)
    for f in glob.glob(os.path.join(conversion_path,"*.csv")):
        os.remove(f)
    for f in glob.glob(os.path.join(pdfs_path,"*.pdf")):
        os.remove(f)
    print("🗑️ Archivos temporales eliminados")

def enviar_api(payload):
    url="https://servimaxinternal.app/etlnodeapp/driverApp/extractBillsCopilatesFromCsv"
    headers={"sede":"key","bogota":"value","Content-Type":"application/json"}
    r=requests.post(url,headers=headers,json=payload)
    if r.status_code==200:
        print("🎉 API devolvió 200")
        limpiar_archivos()
    else:
        print(f"❌ API error {r.status_code}: {r.text}")

# ────────────────────────────────────────────────────────────────
# 9)  PROCESAR UN PAR PLANILLA/DRIVER
# ────────────────────────────────────────────────────────────────
def procesar_par(drv, planilla:str, driver_id:int):
    global planilla_id, idDriver, asesor1, asesor2
    planilla_id = planilla
    idDriver    = driver_id

    asesores=driver_mapping.get(idDriver,["10503","10516"])
    asesor1, asesor2 = map(str, asesores)
    print(f"\n🚚 Planilla {planilla_id}  |  Driver {idDriver}  →  Asesores {asesor1},{asesor2}")

    descargar_pedidos (drv, asesor1,"PAsesorUno")
    descargar_facturas(drv, asesor1,"SAsesorUno")
    descargar_pedidos (drv, asesor2,"PAsesorDos")
    descargar_facturas(drv, asesor2,"SAsesorDos")
    descargar_entregas(drv)

    convertir_a_csv_convertapi()
    pdf_paths = descargar_pdfs(drv)
    payload   = generar_payload(pdf_paths)
    imprimir_payload_truncado(payload)
    enviar_api(payload)

# ────────────────────────────────────────────────────────────────
# 10) MAIN
# ────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    try:
        iniciar_sesion(driver)
        lista = obtener_planillas(driver, limite=20)   # lee las 20 primeras
        for pl, drv_id in lista:
            try:
                procesar_par(driver, pl, drv_id)
            except Exception as e:
                print(f"⚠️  Error en planilla {pl}: {e}")
    finally:
        driver.quit()
        print("👋 Fin de ejecución")


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,927 kB]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,720 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1